# Learn OpenAI Whisper - Chapter 6
## Notebook 2: Using Whisper for podcast transcription using RSS feeds

This notebook provides a example of using FeedParser with OpenAI's Whisper.

[FeedParser](https://feedparser.readthedocs.io/en/latest/) is a Python library for downloading and parsing syndicated feeds including RSS, Atom & RDF Feeds.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1RjVE2zPYY6FFAL3k0S4AgZJYmoV0uH6w)
## Install required Python packages
Run the cell below to install Whisper, FFMPEG (If not installed), and FeedParser.

The Python libraries `openai`, `cohere`, and `tiktoken` are also installed because of dependencies for the `llmx` library. That is because `llmx` relies on them to function correctly.

In [1]:
%%capture
!pip install -q cohere openai tiktoken
!apt-get install ffmpeg
!pip install -q "git+https://github.com/openai/whisper.git"
!pip install -q feedparser requests

In [2]:
import feedparser
import requests
import os
import time
from urllib.parse import urlparse
import subprocess
import re

In [3]:
def list_episodes(feed_url):
    """
    Lists all episodes in the given RSS feed.

    Parameters:
    - feed_url: The RSS feed URL.

    Returns:
    - A list of tuples containing episode titles, their URLs and published dates in 'YYYYMMDD' format.
    """
    d = feedparser.parse(feed_url)

    episodes = []
    for entry in d.entries:
        title = entry.title
        published = time.strftime('%Y%m%d', time.gmtime(time.mktime(entry.published_parsed)))
        url = None
        for link in entry.links:
            if link.type == "audio/mpeg":
                url = link.href
                break
        if url:
            episodes.append((title, url, published))

    return episodes

from urllib.parse import urlparse

def download_episode(url, filename=None):
    """
    Downloads the podcast episode from the given URL.

    Parameters:
    - url: The URL of the podcast episode.
    - filename: The desired filename to save the podcast. If not provided, it'll use the last part of the URL.

    Returns:
    - The path to the downloaded file.
    """
    # If a custom filename is provided, append the appropriate extension from the URL
    if filename:
        parsed_url = urlparse(url)
        # Extract only the base path without any query parameters
        base_path = os.path.basename(parsed_url.path)
        ext = os.path.splitext(base_path)[1]
        filename += ext
    else:
        filename = os.path.basename(parsed_url.path)

    response = requests.get(url, stream=True)
    response.raise_for_status()

    with open(filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    return filename

def download_episode_start_end(url, filename=None, start_at=0, end_at=None):
    """
    Downloads the podcast episode from the given URL and trims it starting from 'start_at' seconds
    and ending at 'end_at' seconds.

    Parameters:
    - url: The URL of the podcast episode.
    - filename: The desired filename to save the podcast. If not provided, it'll use the last part of the URL.
    - start_at: The start time in seconds from where the audio should be trimmed.
    - end_at: The end time in seconds up to which the audio should be trimmed. If not provided or set to 0, the audio will be trimmed till the end.    Returns:
    - The path to the downloaded and trimmed file.
    """
    parsed_url = urlparse(url)
    if filename:
        # Ensure the filename has the correct extension
        ext = os.path.splitext(parsed_url.path)[1]
        filename += ext
    else:
        filename = os.path.basename(parsed_url.path)

    # Download the file
    response = requests.get(url, stream=True)
    response.raise_for_status()
    temp_filename = "temp_" + filename
    with open(temp_filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    # Use ffmpeg to trim the audio file
    trimmed_filename = "trimmed_" + filename
    command = ['ffmpeg', '-y', '-i', temp_filename, '-ss', str(start_at)]
    if end_at is not None and end_at != 0:
        command.extend(['-to', str(end_at)])
    command.extend(['-c', 'copy', trimmed_filename])
    subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Remove the original downloaded file
    os.remove(temp_filename)

    return trimmed_filename

## RSS feed podcast

In [4]:
# Gigantes Podcast Spanish
podcast = 'http://feeds.megaphone.fm/ENTDM3955710136'
# Science, Quickly
# podcast = 'https://feeds.megaphone.fm/SAM8799470961'
# WSJ Minute Briefing
# podcast = 'https://feeds.megaphone.fm/WSJ7928321669'

d = feedparser.parse(podcast)
print(f"Podcast name:", d.feed.title)
print(f"Number of episodes:", len(d.entries))

# List episodes
episodes = list_episodes(podcast)
# Print the first 10 episodes
print("Episodes:")
for idx, (title, url, published) in enumerate(episodes, 1):
    print(f"{idx}. {published}-{title}")
    if idx == 10:
        break

Podcast name: Gigantes Podcast
Number of episodes: 46
Episodes:
1. 20221011-La temporada vino, y se fue
2. 20221004-Beisbol sin barreras con JP Martínez
3. 20220927-El Cometa de Carolina
4. 20220920-Es Septiembre y el roster lo sabe
5. 20220913-El apagón
6. 20220830-La ciencia detrás del juego con Saúl Martínez
7. 20220809-Deje mi corazón en San Francisco
8. 20220726-Yunior Marte, a young veteran
9. 20220719-A la mitad del camino con las miras en alto
10. 20220712-Sube y baja; Las opciones en el béisbol


In [5]:
episode_num = 1 #@param {type:"integer"}
drive_folder = "" #@param {type:"string"}
start_at_seconds = 1300 #@param {type:"integer"}
end_at_seconds = 0 #@param {type:"integer"}

title, url, published = episodes[episode_num - 1]
custom_filename = published + '-' + (re.sub('[^A-Za-z0-9 ]+', '', title[:75]).replace(' ', '_'))

# Download the selected episode
audio_file = download_episode_start_end(url, drive_folder + custom_filename, start_at_seconds, end_at_seconds)
print(f"Downloaded '{title}' as {audio_file}.")

Downloaded 'La temporada vino, y se fue' as trimmed_20221011-La_temporada_vino_y_se_fue.mp3.


In [6]:
import ipywidgets as widgets
widgets.Audio.from_file(audio_file, autoplay=False, loop=False)

Audio(value=b'ID3\x04\x00\x00\x00\x00\x06ETIT2\x00\x00\x00\x1d\x00\x00\x03La temporada vino, y se fue\x00TXXX\…

In [7]:
import whisper
import torch
# NLTK helps to split the transcription sentence by sentence
import nltk
nltk.download('punkt')
from nltk import sent_tokenize

model = whisper.load_model("small")

audio = whisper.load_audio(audio_file)
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
audio_lang = max(probs, key=probs.get)
print(f"Detected language: {audio_lang}")

# decode the audio
options = whisper.DecodingOptions(fp16=torch.cuda.is_available(), language=audio_lang, task='transcribe')
result = whisper.decode(model, mel, options)

# print the recognized text
print("----\nTranscription from audio:")
for sent in sent_tokenize(result.text):
  print(sent)

# decode the audio
options = whisper.DecodingOptions(fp16=torch.cuda.is_available(), language=audio_lang, task='translate')
result = whisper.decode(model, mel, options)

# print the recognized text
print("----\nTranslation from audio:")
for sent in sent_tokenize(result.text):
  print(sent)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
100%|███████████████████████████████████████| 461M/461M [00:08<00:00, 57.6MiB/s]


Detected language: es
----
Transcription from audio:
y el bullpen estuviese en una mejor posición y yo entiendo que mirando hacia el futuro vamos a ver cómo vamos a tener alternativa de una manera positiva para que el equipo cuando sufra esas lesiones aún estén en una buena posición.
Y qué bueno que mencionas eso del Open Air, cómo hacerle quizás la vida más simple a los jovencitos que vienen porque muchos piensan que el usar el Open Air es por diseño, pero no es por necesidad que los equipos en ocasiones
----
Translation from audio:
The bullpen is in a better position and I understand that looking at the future we will see how we will have an alternative in a positive way so that the team, when suffering those injuries, is still in a good position.
And how good is that you mention the opener, how to make perhaps the simplest life for the young people who come because many think that using the opener is for design, but it is not because of the need that the teams on occasion
